# Introduction



---



---



In this notebook, we will implement the following tests for the NONLINEAR KOPTEVA PROBLEM in DEFCON:



Inside epsilon continuation:

- Gauss Siedal MP-Iteration (Fieldsplit Code). Compute Hessian Mesh Density Function based on the current physical solution and mesh, take $\alpha_{hat} = \epsilon^{-3/4}$ (for $m=0$) or $\alpha_{hat} = \epsilon^{-5/4}$ (for $m=1$) for the scaling on the Hessian in the Mesh Density Function M.


- Tolerance for Mp-Iteration is (1/10)*epsilon (based on previous testing)



---



To see the manual version of the code that gets put into Defcon see: Cleaned_EpsilonBasedMeshTol_EpsilonScaledAlpha_GaussSiedal_MPIteration_KoptevaProblem.ipynb




# Results and Conclusions


---



---

- For the solutions I do compute, meshes and the solutions look good.

- However, I compute many duplicate solutions causing me to stall at large values of epsilon

- We think we need a more informative squarednorm to go into the Deflation Operator.

- Right now, for the norm, we have:

      inner(diffu, diffu)*dx + (eps**2)*inner(grad(diffu), grad(diffu))*dx + inner(diffx, diffx)*dx
 We think this is causing these duplicate solutions because this norm is taken in computational coordinates and not physical coordinates. In computational coordinates, two physical solutions and meshes which look similar (in physical coordinate land) can be just differant enough to have large norms of their differences.

- Also, we could have the same solution (ish) with slightly different meshes but the mesh norm will cause this squarted norm to be large.

- We think, instead, we want some version of the following for the norm:

        uz_norm = assemble( uz * uz * xz.dx(0) * dx )
        uw_norm = assemble( uw * uw * xw.dx(0) * dx )
        u_norm_diff = (uz_norm - uw_norm)**2
where we only monitor the differences in the physical solutions and we do so in their respective physical coordinates (jacobian transformations in these integrals bring us from computational to physical coordinate land)

- We started working on this updated norm but we didnt get it done

- Another thing to experiment with is a larger power parameter in the Deflation operator. Right now, I have power = 3 and shift = 1.

- Link to Defcon Bitbucket: https://bitbucket.org/maggiemcc02/defcon/src/master/


# Install Firedrake and Import Librairies

In [12]:
# install firedrake

# hide output
%%capture

# try:
#     import firedrake
# except ImportError:
#     !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
#     import firedrake


try:
    from firedrake import *
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-release-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    from firedrake import *

In [13]:
# Code in this cell makes plots appear an appropriate size and resolution in the browser window

%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (11, 6)

In [14]:
# import firedrake tools
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt # firedrake makes use of matplotlib tools
from firedrake.pyplot import tripcolor, tricontour, triplot #firedrake plotting
from IPython.display import display
from mpl_toolkits import mplot3d
import scipy as sci



# Install My Version of Defcon

In [15]:
!pip install git+https://bitbucket.org/maggiemcc02/defcon/src/master/

  Cloning https://bitbucket.org/maggiemcc02/defcon/src/master/ to /tmp/pip-req-build-uwkl3ji6
  Running command git clone --filter=blob:none --quiet https://bitbucket.org/maggiemcc02/defcon/src/master/ /tmp/pip-req-build-uwkl3ji6
  Resolved https://bitbucket.org/maggiemcc02/defcon/src/master/ to commit d431432e4b4d897900a79a1c2160100468cae40a
  Preparing metadata (setup.py) ... done


In [16]:
!git clone https://bitbucket.org/maggiemcc02/defcon/src/master/

fatal: destination path 'master' already exists and is not an empty directory.


In [17]:
# try to just run bratu

# !python master/examples/firedrake/bratu/bratu.py

# 1D Kopteva Problem

In [18]:
# try to address MPI issue
from mpi4py import MPI
from defcon import *

!mpiexec --map-by :OVERSUBSCRIBE -n 4  --allow-run-as-root  python master/examples/firedrake/KoptevaMPIter/Kopteva_Attempt1_Aug2025.py

# !mpiexec --map-by :OVERSUBSCRIBE -n 4  --allow-run-as-root  python master/examples/firedrake/linear_1D/linear_1D.py


[15-08-25 @ 16:25:40] MASTER:   Using user-supplied initial guesses at (np.float64(0.1),)
[15-08-25 @ 16:25:40] MASTER:   Sending task DeflationTask(taskid=2, oldparams=None, freeindex=0, branchid=2, newparams=(np.float64(0.1),)) to team 0
[15-08-25 @ 16:25:40] MASTER:   Memory used: 334344
[15-08-25 @ 16:25:40] MASTER:   Sending task DeflationTask(taskid=3, oldparams=None, freeindex=0, branchid=3, newparams=(np.float64(0.1),)) to team 1
[15-08-25 @ 16:25:40] MASTER:   Memory used: 334344
[15-08-25 @ 16:25:40] MASTER:   Sending task DeflationTask(taskid=9, oldparams=None, freeindex=0, branchid=9, newparams=(np.float64(0.1),)) to team 2
[15-08-25 @ 16:25:40] MASTER:   Memory used: 334344
[15-08-25 @ 16:25:40] MASTER:   Cannot dispatch any tasks, waiting for response.
[15-08-25 @ 16:25:40] TEAM   1: Fetching task
[15-08-25 @ 16:25:40] TEAM   1: Executing task DeflationTask(taskid=3, oldparams=None, freeindex=0, branchid=3, newparams=(np.float64(0.1),))
[15-08-25 @ 16:25:40] TEAM   2: Fet

# Save the Desired Solutions

In [19]:
# from google.colab import drive
# drive.mount('/content/drive')

In [20]:
# !cp -r /content/output/epsilon=1.000000000000000e-02 /content/drive/MyDrive/

In [21]:
# !cp -r /content/output/epsilon=1.000000000000000e-01 /content/drive/MyDrive/

In [22]:
# !cp -r /content/output/epsilon=1.778279410038923e-03 /content/drive/MyDrive/